Importando os dados de treino

In [2]:
import pandas as pd
treino = pd.read_csv('/content/train.csv')

In [2]:
treino

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


Removendo as linhas com dados faltantes

In [3]:
colunas = ['Survived','PassengerId','Pclass','Sex','Age','SibSp','Parch','Ticket','Fare','Embarked']
x_treino=treino.dropna(subset=colunas)[colunas]

In [93]:
x_treino.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Survived     712 non-null    int64  
 1   PassengerId  712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Sex          712 non-null    object 
 4   Age          712 non-null    float64
 5   SibSp        712 non-null    int64  
 6   Parch        712 non-null    int64  
 7   Ticket       712 non-null    object 
 8   Fare         712 non-null    float64
 9   Embarked     712 non-null    object 
dtypes: float64(2), int64(5), object(3)
memory usage: 61.2+ KB


Removendo a coluna 'Survived' do treino

In [4]:
y_treino=x_treino['Survived']
x_treino=x_treino.drop('Survived',axis=1)

Importando os dados de teste

In [5]:
teste = pd.read_csv('/content/test.csv')
y_teste=pd.read_csv('/content/gender_submission.csv')

In [99]:
teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
 11  Survived     418 non-null    int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [6]:
teste['Survived']=y_teste['Survived']

Removendo dados faltantes

In [7]:
x_teste=teste.dropna(subset=colunas)[colunas]

In [8]:
y_teste=x_teste['Survived']
x_teste=x_teste.drop('Survived',axis=1)

Transformando variáveis categóricas em numéricas

In [9]:
from sklearn.preprocessing import LabelEncoder
le_ticket= LabelEncoder()
le_ticket.fit(pd.concat([x_treino,x_teste])['Ticket'].unique())
x_treino['Ticket']=le_ticket.transform(x_treino['Ticket'])
x_teste['Ticket']=le_ticket.transform(x_teste['Ticket'])

In [10]:
le_sex= LabelEncoder()
le_sex.fit(pd.concat([x_treino,x_teste])['Sex'].unique())
x_treino['Sex']=le_sex.transform(x_treino['Sex'])
x_teste['Sex']=le_sex.transform(x_teste['Sex'])

In [11]:
le_embarked= LabelEncoder()
le_embarked.fit(pd.concat([x_treino,x_teste])['Embarked'].unique())
x_treino['Embarked']=le_embarked.transform(x_treino['Embarked'])
x_teste['Embarked']=le_embarked.transform(x_teste['Embarked'])

Modelo Árvore de decisão

In [45]:
from sklearn.tree import DecisionTreeClassifier
model_arvore=DecisionTreeClassifier(max_depth=3)
model_arvore.fit(x_treino,y_treino)

DecisionTreeClassifier(max_depth=3)

In [46]:
y_decisiontree=model_arvore.predict(x_teste)


In [47]:
from sklearn.metrics import accuracy_score
acuracia_decisiontree = accuracy_score(y_teste,y_decisiontree)

Modelo Random Forest

In [53]:
from sklearn.ensemble import RandomForestClassifier
model_random = RandomForestClassifier(max_depth=3)
model_random.fit(x_treino,y_treino)

RandomForestClassifier(max_depth=3)

In [54]:
y_randomforest=model_random.predict(x_teste)

In [55]:
acuracia_randomforest = accuracy_score(y_teste,y_randomforest)

Comparação da acurácia dos modelos

In [56]:
print(f'Decision Tree:{acuracia_decisiontree}')
print(f'Random Forest:{acuracia_randomforest}')

Decision Tree:0.9667673716012085
Random Forest:0.8489425981873112
